In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Modelos
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Métricas
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, 
    f1_score, confusion_matrix, classification_report, roc_curve,
    precision_recall_curve, average_precision_score
)

import pickle
from datetime import datetime

print("="*80)
print("MODELADO MACHINE LEARNING - PREDICCIÓN DE ANEMIA INFANTIL")
print("ENDES 2015-2024 | ESTRATIFICACIÓN CRUZADA AÑO × ANEMIA")
print("="*80)

# ============================================================
# CONFIGURACIÓN
# ============================================================
ruta_datos = Path(r"D:\Bases_train_test")
ruta_resultados = Path(r"D:\Resultados_ML")
ruta_resultados.mkdir(exist_ok=True)

# ============================================================
# 1. CARGAR DATOS
# ============================================================
print("\n" + "="*80)
print("1. CARGANDO DATOS")
print("="*80)

print("\nCargando datasets...")
df_train = pd.read_csv(ruta_datos / "endes_train_2015_2024.csv", encoding='utf-8-sig')
df_test = pd.read_csv(ruta_datos / "endes_test_2015_2024.csv", encoding='utf-8-sig')
df_dev = pd.read_csv(ruta_datos / "endes_dev_2015_2024.csv", encoding='utf-8-sig')

print(f"\n✓ TRAIN: {len(df_train):,} registros × {df_train.shape[1]} columnas")
print(f"   Años: {sorted(df_train['ANIO'].unique())}")
print(f"   Prevalencia anemia: {df_train['ANEMIA'].mean()*100:.2f}%")

print(f"\n✓ TEST:  {len(df_test):,} registros × {df_test.shape[1]} columnas")
print(f"   Años: {sorted(df_test['ANIO'].unique())}")
print(f"   Prevalencia anemia: {df_test['ANEMIA'].mean()*100:.2f}%")

print(f"\n✓ DEV:   {len(df_dev):,} registros × {df_dev.shape[1]} columnas")
print(f"   Años: {sorted(df_dev['ANIO'].unique())}")
print(f"   Prevalencia anemia: {df_dev['ANEMIA'].mean()*100:.2f}%")

# ============================================================
# 2. ANÁLISIS EXPLORATORIO INICIAL
# ============================================================
print("\n" + "="*80)
print("2. ANÁLISIS EXPLORATORIO")
print("="*80)

print(f"\n📊 Distribución de ANEMIA por partición:")
print(f"\n{'Partición':<10} {'Sin anemia':>12} {'Con anemia':>12} {'Total':>10} {'% Anemia':>10}")
print("-" * 60)

for nombre, df in [('TRAIN', df_train), ('TEST', df_test), ('DEV', df_dev)]:
    sin_anemia = (df['ANEMIA'] == 0).sum()
    con_anemia = (df['ANEMIA'] == 1).sum()
    total = len(df)
    pct = con_anemia / total * 100
    print(f"{nombre:<10} {sin_anemia:>12,} {con_anemia:>12,} {total:>10,} {pct:>9.2f}%")

# Verificar columnas disponibles
print(f"\n📋 Columnas disponibles: {df_train.shape[1]}")
print(f"   Primeras 10: {df_train.columns.tolist()[:10]}")

# ============================================================
# 3. SELECCIÓN Y PREPARACIÓN DE FEATURES
# ============================================================
print("\n" + "="*80)
print("3. SELECCIÓN Y PREPARACIÓN DE FEATURES")
print("="*80)

# Definir features disponibles
features_numericas = [
    'HC1',      # Edad en meses
    # 'HC53',   # ❌ EXCLUIDO - Hemoglobina (DATA LEAKAGE - define anemia)
    'HC70',     # Altura/edad Z-score
    #'HV009',    # Miembros del hogar
    'HV040',    # Altitud
   # 'HV271',    # Índice de riqueza
   # 'HW1',      # Edad en meses (REC44)
   # 'HW2',      # Peso
    'HW3',      # Talla
   # 'HW70',     # Talla/Edad Z-score
  #  'HW71',     # Peso/Edad Z-score
  #  'HW72',     # Peso/Talla Z-score
  #  'HW73',     # IMC Z-score
    'BORD',     # Orden de nacimiento
    'V012',     # Edad de la madre
    'V133',     # Educación en años
   # 'ANIO',     # Año (para capturar tendencias temporales)
]

features_categoricas = [
    'HC27',     # Sexo
  #  'HC55',     # Resultado de medición
   # 'HC57',     # Nivel de anemia (aunque está relacionado con target)
    'HV024',    # Región/departamento
    'HV025',    # Área de residencia
  #  'HV201',    # Fuente de agua
  #  'HV205',    # Tipo de servicio sanitario
  #  'HV206',    # Electricidad
    'HV237',    # Tratamiento del agua
   # 'V025',     # Área de residencia (madre)
    'V106',     # Nivel educativo madre
    'V190',     # Índice de riqueza (categorizado)
]

# Verificar qué features están disponibles
features_numericas_disponibles = [f for f in features_numericas if f in df_train.columns]
features_categoricas_disponibles = [f for f in features_categoricas if f in df_train.columns]

# EXCLUIR HC57 porque es derivado directo del target
if 'HC57' in features_categoricas_disponibles:
    features_categoricas_disponibles.remove('HC57')
    print(f"\n⚠️  HC57 excluido (derivado directo del target)")

print(f"\n✓ Features numéricas disponibles: {len(features_numericas_disponibles)}")
print(f"   {features_numericas_disponibles}")

print(f"\n✓ Features categóricas disponibles: {len(features_categoricas_disponibles)}")
print(f"   {features_categoricas_disponibles}")

# Combinar features
features = features_numericas_disponibles + features_categoricas_disponibles
print(f"\n✓ Total features: {len(features)}")

# ============================================================
# 4. PREPROCESAMIENTO
# ============================================================
print("\n" + "="*80)
print("4. PREPROCESAMIENTO")
print("="*80)

def preprocesar_datos(df, features_num, features_cat, le_dict=None, fit=True):
    """
    Preprocesa los datos:
    - Imputa missings en numéricas con mediana
    - Codifica categóricas con LabelEncoder
    """
    df_prep = df.copy()
    
    # 1. Imputar numéricas con mediana
    for col in features_num:
        if col in df_prep.columns:
            if df_prep[col].isna().any():
                mediana = df_prep[col].median()
                df_prep[col].fillna(mediana, inplace=True)
    
    # 2. Codificar categóricas
    if le_dict is None:
        le_dict = {}
    
    for col in features_cat:
        if col in df_prep.columns:
            # Convertir a string y manejar missings
            df_prep[col] = df_prep[col].astype(str).fillna('missing')
            
            if fit:
                # Crear y ajustar LabelEncoder
                le = LabelEncoder()
                df_prep[col] = le.fit_transform(df_prep[col])
                le_dict[col] = le
            else:
                # Usar LabelEncoder existente
                le = le_dict[col]
                # Manejar categorías no vistas
                df_prep[col] = df_prep[col].apply(
                    lambda x: x if x in le.classes_ else 'missing'
                )
                # Añadir 'missing' si no está en classes_
                if 'missing' not in le.classes_:
                    le.classes_ = np.append(le.classes_, 'missing')
                df_prep[col] = le.transform(df_prep[col])
    
    return df_prep, le_dict

# Preprocesar TRAIN
print("\nPreprocesando TRAIN...")
df_train_prep, label_encoders = preprocesar_datos(
    df_train, 
    features_numericas_disponibles, 
    features_categoricas_disponibles,
    fit=True
)

# Preprocesar TEST
print("Preprocesando TEST...")
df_test_prep, _ = preprocesar_datos(
    df_test,
    features_numericas_disponibles,
    features_categoricas_disponibles,
    le_dict=label_encoders,
    fit=False
)

# Preprocesar DEV
print("Preprocesando DEV...")
df_dev_prep, _ = preprocesar_datos(
    df_dev,
    features_numericas_disponibles,
    features_categoricas_disponibles,
    le_dict=label_encoders,
    fit=False
)

# Extraer X e y
X_train = df_train_prep[features]
y_train = df_train_prep['ANEMIA']

X_test = df_test_prep[features]
y_test = df_test_prep['ANEMIA']

X_dev = df_dev_prep[features]
y_dev = df_dev_prep['ANEMIA']

# Usar pesos muestrales si está disponible
if 'PESO' in df_train.columns:
    # Limpiar NaN en pesos y normalizar
    w_train = df_train['PESO'].fillna(df_train['PESO'].median())
    w_test = df_test['PESO'].fillna(df_test['PESO'].median())
    w_dev = df_dev['PESO'].fillna(df_dev['PESO'].median())
    
    # Verificar que no haya NaN
    if w_train.isna().any() or w_test.isna().any() or w_dev.isna().any():
        print(f"\n⚠️  Pesos con NaN después de imputación, no se usarán pesos")
        w_train = None
        w_test = None
        w_dev = None
        usar_pesos = False
    else:
        usar_pesos = True
        print(f"\n✓ Usando pesos muestrales (variable PESO)")
        print(f"   Rango TRAIN: {w_train.min():.6f} - {w_train.max():.6f}")
else:
    w_train = None
    w_test = None
    w_dev = None
    usar_pesos = False
    print(f"\n⚠️  No se encontró variable PESO, no se usarán pesos muestrales")

print(f"\n✓ Preparación completada:")
print(f"   X_train: {X_train.shape}")
print(f"   X_test:  {X_test.shape}")
print(f"   X_dev:   {X_dev.shape}")

# Verificar missings
print(f"\n🔍 Verificando missings en X_train:")
missings = X_train.isna().sum().sum()
if missings > 0:
    print(f"   ⚠️  {missings} missings encontrados")
    print(X_train.isna().sum()[X_train.isna().sum() > 0])
else:
    print(f"   ✓ Sin missings")

# ============================================================
# 5. DEFINICIÓN DE MODELOS
# ============================================================
print("\n" + "="*80)
print("5. DEFINICIÓN DE MODELOS")
print("="*80)

modelos = {
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=20,
        min_samples_leaf=10,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    ),
    
    'XGBoost': XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
    ),
    
    'LightGBM': LGBMClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced',
        verbosity=-1
    )
}

print(f"\n✓ Modelos definidos:")
for nombre in modelos.keys():
    print(f"   • {nombre}")

# ============================================================
# 6. ENTRENAMIENTO Y EVALUACIÓN
# ============================================================
print("\n" + "="*80)
print("6. ENTRENAMIENTO Y EVALUACIÓN")
print("="*80)

def evaluar_modelo(y_true, y_pred, y_pred_proba, conjunto, peso=None):
    """Calcula métricas de evaluación"""
    
    # Si hay pesos, calcular métricas ponderadas
    sample_weight = peso if peso is not None else None
    
    metricas = {
        'Conjunto': conjunto,
        'Accuracy': accuracy_score(y_true, y_pred, sample_weight=sample_weight),
        'Precision': precision_score(y_true, y_pred, sample_weight=sample_weight, zero_division=0),
        'Recall': recall_score(y_true, y_pred, sample_weight=sample_weight, zero_division=0),
        'F1': f1_score(y_true, y_pred, sample_weight=sample_weight, zero_division=0),
        'AUC': roc_auc_score(y_true, y_pred_proba, sample_weight=sample_weight),
        'AP': average_precision_score(y_true, y_pred_proba, sample_weight=sample_weight)
    }
    
    return metricas

resultados = []
modelos_entrenados = {}

for nombre_modelo, modelo in modelos.items():
    print(f"\n{'='*80}")
    print(f"Entrenando: {nombre_modelo}")
    print(f"{'='*80}")
    
    # Entrenar
    print(f"\n⏳ Entrenando en TRAIN...")
    if usar_pesos:
        modelo.fit(X_train, y_train, sample_weight=w_train)
    else:
        modelo.fit(X_train, y_train)
    
    # Guardar modelo entrenado
    modelos_entrenados[nombre_modelo] = modelo
    
    # Predicciones en TRAIN
    print(f"📊 Evaluando en TRAIN...")
    y_train_pred = modelo.predict(X_train)
    y_train_proba = modelo.predict_proba(X_train)[:, 1]
    metricas_train = evaluar_modelo(y_train, y_train_pred, y_train_proba, 'TRAIN', w_train if usar_pesos else None)
    metricas_train['Modelo'] = nombre_modelo
    
    # Predicciones en TEST
    print(f"📊 Evaluando en TEST...")
    y_test_pred = modelo.predict(X_test)
    y_test_proba = modelo.predict_proba(X_test)[:, 1]
    metricas_test = evaluar_modelo(y_test, y_test_pred, y_test_proba, 'TEST', w_test if usar_pesos else None)
    metricas_test['Modelo'] = nombre_modelo
    
    # Predicciones en DEV
    print(f"📊 Evaluando en DEV...")
    y_dev_pred = modelo.predict(X_dev)
    y_dev_proba = modelo.predict_proba(X_dev)[:, 1]
    metricas_dev = evaluar_modelo(y_dev, y_dev_pred, y_dev_proba, 'DEV', w_dev if usar_pesos else None)
    metricas_dev['Modelo'] = nombre_modelo
    
    # Agregar resultados
    resultados.extend([metricas_train, metricas_test, metricas_dev])
    
    # Mostrar resumen
    print(f"\n✓ {nombre_modelo} - Resumen:")
    print(f"   TRAIN → AUC: {metricas_train['AUC']:.4f}, F1: {metricas_train['F1']:.4f}")
    print(f"   TEST  → AUC: {metricas_test['AUC']:.4f}, F1: {metricas_test['F1']:.4f}")
    print(f"   DEV   → AUC: {metricas_dev['AUC']:.4f}, F1: {metricas_dev['F1']:.4f}")

# ============================================================
# 7. COMPARACIÓN DE MODELOS
# ============================================================
print("\n" + "="*80)
print("7. COMPARACIÓN DE MODELOS")
print("="*80)

df_resultados = pd.DataFrame(resultados)

# Pivot para mejor visualización
df_pivot = df_resultados.pivot_table(
    index='Modelo',
    columns='Conjunto',
    values=['AUC', 'F1', 'Accuracy', 'Precision', 'Recall']
)

print(f"\n📊 Tabla comparativa de modelos:")
print(df_pivot.round(4))

# Identificar mejor modelo (por AUC en DEV)
mejor_modelo_nombre = df_resultados[df_resultados['Conjunto'] == 'DEV'].sort_values('AUC', ascending=False).iloc[0]['Modelo']
print(f"\n🏆 Mejor modelo (por AUC en DEV): {mejor_modelo_nombre}")

mejor_modelo = modelos_entrenados[mejor_modelo_nombre]

# ============================================================
# 8. ANÁLISIS DETALLADO DEL MEJOR MODELO
# ============================================================
print("\n" + "="*80)
print(f"8. ANÁLISIS DETALLADO: {mejor_modelo_nombre}")
print("="*80)

# Matriz de confusión en DEV
y_dev_pred_mejor = mejor_modelo.predict(X_dev)
cm = confusion_matrix(y_dev, y_dev_pred_mejor)

print(f"\n📊 Matriz de Confusión (DEV):")
print(f"\n                 Predicho: Sin   Predicho: Con")
print(f"Real: Sin anemia    {cm[0,0]:>6}        {cm[0,1]:>6}")
print(f"Real: Con anemia    {cm[1,0]:>6}        {cm[1,1]:>6}")

print(f"\n📈 Classification Report (DEV):")
print(classification_report(y_dev, y_dev_pred_mejor, target_names=['Sin anemia', 'Con anemia']))

# Feature importance (si el modelo lo soporta)
if hasattr(mejor_modelo, 'feature_importances_'):
    print(f"\n🔝 Top 15 Features más importantes:")
    importances = pd.DataFrame({
        'Feature': features,
        'Importance': mejor_modelo.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(importances.head(15).to_string(index=False))

# ============================================================
# 9. VISUALIZACIONES
# ============================================================
print("\n" + "="*80)
print("9. GENERANDO VISUALIZACIONES")
print("="*80)

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparación de AUC por modelo
ax = axes[0, 0]
df_auc = df_resultados.pivot(index='Modelo', columns='Conjunto', values='AUC')
df_auc.plot(kind='bar', ax=ax, rot=0)
ax.set_title('Comparación AUC por Modelo y Conjunto', fontsize=14, fontweight='bold')
ax.set_ylabel('AUC', fontsize=12)
ax.set_xlabel('Modelo', fontsize=12)
ax.legend(title='Conjunto', fontsize=10)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0.5, 1.0])

# 2. Comparación de F1 por modelo
ax = axes[0, 1]
df_f1 = df_resultados.pivot(index='Modelo', columns='Conjunto', values='F1')
df_f1.plot(kind='bar', ax=ax, rot=0)
ax.set_title('Comparación F1-Score por Modelo y Conjunto', fontsize=14, fontweight='bold')
ax.set_ylabel('F1-Score', fontsize=12)
ax.set_xlabel('Modelo', fontsize=12)
ax.legend(title='Conjunto', fontsize=10)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.0])

# 3. Curva ROC del mejor modelo
ax = axes[1, 0]
for conjunto, X, y, color in [('TRAIN', X_train, y_train, 'blue'),
                               ('TEST', X_test, y_test, 'orange'),
                               ('DEV', X_dev, y_dev, 'green')]:
    y_proba = mejor_modelo.predict_proba(X)[:, 1]
    fpr, tpr, _ = roc_curve(y, y_proba)
    auc_val = roc_auc_score(y, y_proba)
    ax.plot(fpr, tpr, label=f'{conjunto} (AUC={auc_val:.3f})', color=color, linewidth=2)

ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
ax.set_title(f'Curva ROC - {mejor_modelo_nombre}', fontsize=14, fontweight='bold')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

# 4. Matriz de confusión del mejor modelo en DEV
ax = axes[1, 1]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
            xticklabels=['Sin anemia', 'Con anemia'],
            yticklabels=['Sin anemia', 'Con anemia'],
            cbar_kws={'label': 'Cantidad'})
ax.set_title(f'Matriz de Confusión (DEV) - {mejor_modelo_nombre}', fontsize=14, fontweight='bold')
ax.set_ylabel('Real', fontsize=12)
ax.set_xlabel('Predicho', fontsize=12)

plt.tight_layout()
archivo_grafico = ruta_resultados / f"comparacion_modelos_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
plt.savefig(archivo_grafico, dpi=300, bbox_inches='tight')
print(f"\n✓ Gráficos guardados: {archivo_grafico}")
plt.close()

# Gráfico adicional: Feature Importance
if hasattr(mejor_modelo, 'feature_importances_'):
    fig, ax = plt.subplots(figsize=(10, 8))
    importances_top20 = importances.head(20)
    ax.barh(importances_top20['Feature'], importances_top20['Importance'])
    ax.set_xlabel('Importancia', fontsize=12)
    ax.set_title(f'Top 20 Features - {mejor_modelo_nombre}', fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    plt.tight_layout()
    archivo_importance = ruta_resultados / f"feature_importance_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    plt.savefig(archivo_importance, dpi=300, bbox_inches='tight')
    print(f"✓ Feature importance guardado: {archivo_importance}")
    plt.close()

# ============================================================
# 10. GUARDAR RESULTADOS
# ============================================================
print("\n" + "="*80)
print("10. GUARDANDO RESULTADOS")
print("="*80)

# Guardar tabla de resultados
archivo_resultados = ruta_resultados / f"resultados_modelos_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

with pd.ExcelWriter(archivo_resultados, engine='openpyxl') as writer:
    # Hoja 1: Resultados detallados
    df_resultados.to_excel(writer, sheet_name='Resultados_Detallados', index=False)
    
    # Hoja 2: Comparación (pivot)
    df_pivot.to_excel(writer, sheet_name='Comparacion')
    
    # Hoja 3: Feature importance (si existe)
    if hasattr(mejor_modelo, 'feature_importances_'):
        importances.to_excel(writer, sheet_name='Feature_Importance', index=False)
    
    # Hoja 4: Confusion Matrix
    cm_df = pd.DataFrame(cm, 
                         index=['Real: Sin anemia', 'Real: Con anemia'],
                         columns=['Pred: Sin anemia', 'Pred: Con anemia'])
    cm_df.to_excel(writer, sheet_name='Confusion_Matrix')

print(f"\n✓ Resultados guardados: {archivo_resultados}")

# Guardar mejor modelo
archivo_modelo = ruta_resultados / f"mejor_modelo_{mejor_modelo_nombre.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
with open(archivo_modelo, 'wb') as f:
    pickle.dump({
        'modelo': mejor_modelo,
        'label_encoders': label_encoders,
        'features': features,
        'features_numericas': features_numericas_disponibles,
        'features_categoricas': features_categoricas_disponibles
    }, f)

print(f"✓ Mejor modelo guardado: {archivo_modelo}")

# ============================================================
# RESUMEN FINAL
# ============================================================
print("\n" + "="*80)
print("✅ PROCESO COMPLETADO")
print("="*80)

print(f"\n📁 Archivos generados en: {ruta_resultados}")
print(f"   1. {archivo_resultados.name}")
print(f"   2. {archivo_grafico.name}")
if hasattr(mejor_modelo, 'feature_importances_'):
    print(f"   3. {archivo_importance.name}")
print(f"   4. {archivo_modelo.name}")

print(f"\n🏆 Resumen:")
print(f"   Mejor modelo: {mejor_modelo_nombre}")
metricas_dev_mejor = df_resultados[(df_resultados['Modelo'] == mejor_modelo_nombre) & 
                                   (df_resultados['Conjunto'] == 'DEV')].iloc[0]
print(f"   Métricas en DEV:")
print(f"      AUC:       {metricas_dev_mejor['AUC']:.4f}")
print(f"      F1-Score:  {metricas_dev_mejor['F1']:.4f}")
print(f"      Accuracy:  {metricas_dev_mejor['Accuracy']:.4f}")
print(f"      Precision: {metricas_dev_mejor['Precision']:.4f}")
print(f"      Recall:    {metricas_dev_mejor['Recall']:.4f}")

print(f"\n💡 Próximos pasos:")
print(f"   1. Revisar archivo Excel con resultados detallados")
print(f"   2. Analizar feature importance para interpretabilidad")
print(f"   3. Considerar ajuste de hiperparámetros si es necesario")
print(f"   4. Validar resultados con expertos en salud pública")

print("\n" + "="*80)

MODELADO MACHINE LEARNING - PREDICCIÓN DE ANEMIA INFANTIL
ENDES 2015-2024 | ESTRATIFICACIÓN CRUZADA AÑO × ANEMIA

1. CARGANDO DATOS

Cargando datasets...

✓ TRAIN: 92,030 registros × 43 columnas
   Años: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
   Prevalencia anemia: 46.33%

✓ TEST:  26,291 registros × 43 columnas
   Años: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
   Prevalencia anemia: 46.33%

✓ DEV:   13,147 registros × 43 columnas
   Años: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
   Prevalencia anemia: 46.32%

2. ANÁLISIS EXPLORATORIO

📊 Distribución de ANEMIA por partición:

Partición    Sin anemia   Con anemia      Total   % Anemia
------------------------------------------------------------
TRAIN            49,391       42,639     92,030     46.33%
TEST             14,111       12,180     26,291     46.33%
DEV               7,057        6,090     13,147     46.32%

📋 Columnas disponibles: 43
   Primeras 10: ['HHID', 'HC0'

In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    roc_auc_score, f1_score, classification_report, 
    confusion_matrix, make_scorer, accuracy_score,
    recall_score, precision_score
)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import pickle
from datetime import datetime

print("="*80)
print("OPTIMIZACIÓN AVANZADA - PREDICCIÓN DE ANEMIA INFANTIL")
print("Sin HC53 (hemoglobina) para evitar data leakage")
print("Versión Corregida con Ajuste Automático de SMOTE")
print("="*80)

# ============================================================
# CONFIGURACIÓN
# ============================================================
ruta_datos = Path(r"D:\Bases_train_test")
ruta_resultados = Path(r"D:\Resultados_ML\Optimizado")
ruta_resultados.mkdir(exist_ok=True, parents=True)

# ============================================================
# 1. CARGAR Y PREPARAR DATOS
# ============================================================
print("\n1. CARGANDO DATOS...")

df_train = pd.read_csv(ruta_datos / "endes_train_2015_2024.csv")
df_test = pd.read_csv(ruta_datos / "endes_test_2015_2024.csv")
df_dev = pd.read_csv(ruta_datos / "endes_dev_2015_2024.csv")

# Features (SIN HC53 ni ANIO)
features_numericas = [
    'HC1', 'HC70', 'HV009', 'HV040', 'HV271',
    'HW1', 'HW2', 'HW3',
    'BORD', 'V012', 'V133'
]

features_categoricas = [
    'HC27', 'HC55', 'HV024', 'HV025', 'HV237',
    'V025', 'V106', 'V190'
]

# Preprocesamiento
def preprocesar(df, features_num, features_cat, le_dict=None, fit=True):
    df_prep = df.copy()
    
    # Imputar numéricas
    for col in features_num:
        if col in df_prep.columns:
            df_prep[col].fillna(df_prep[col].median(), inplace=True)
    
    # Codificar categóricas
    if le_dict is None:
        le_dict = {}
    
    for col in features_cat:
        if col in df_prep.columns:
            df_prep[col] = df_prep[col].astype(str).fillna('missing')
            
            if fit:
                le = LabelEncoder()
                df_prep[col] = le.fit_transform(df_prep[col])
                le_dict[col] = le
            else:
                le = le_dict[col]
                df_prep[col] = df_prep[col].apply(
                    lambda x: x if x in le.classes_ else 'missing'
                )
                if 'missing' not in le.classes_:
                    le.classes_ = np.append(le.classes_, 'missing')
                df_prep[col] = le.transform(df_prep[col])
    
    return df_prep, le_dict

features_num_disp = [f for f in features_numericas if f in df_train.columns]
features_cat_disp = [f for f in features_categoricas if f in df_train.columns]
features = features_num_disp + features_cat_disp

df_train_prep, label_encoders = preprocesar(df_train, features_num_disp, features_cat_disp, fit=True)
df_test_prep, _ = preprocesar(df_test, features_num_disp, features_cat_disp, le_dict=label_encoders, fit=False)
df_dev_prep, _ = preprocesar(df_dev, features_num_disp, features_cat_disp, le_dict=label_encoders, fit=False)

X_train = df_train_prep[features]
y_train = df_train_prep['ANEMIA']
X_test = df_test_prep[features]
y_test = df_test_prep['ANEMIA']
X_dev = df_dev_prep[features]
y_dev = df_dev_prep['ANEMIA']

# Pesos
w_train = df_train['PESO'].fillna(df_train['PESO'].median()) if 'PESO' in df_train.columns else None
w_test = df_test['PESO'].fillna(df_test['PESO'].median()) if 'PESO' in df_test.columns else None
w_dev = df_dev['PESO'].fillna(df_dev['PESO'].median()) if 'PESO' in df_dev.columns else None

# Análisis de distribución de clases
print(f"\n✓ Datos preparados: {len(features)} features")
print(f"   X_train: {X_train.shape}")
print(f"\n📊 Distribución de clases en TRAIN:")
class_counts = y_train.value_counts()
print(f"   Clase 0 (Sin anemia): {class_counts[0]:,} ({class_counts[0]/len(y_train)*100:.2f}%)")
print(f"   Clase 1 (Con anemia): {class_counts[1]:,} ({class_counts[1]/len(y_train)*100:.2f}%)")

ratio_actual = class_counts[1] / class_counts[0]
print(f"   Ratio actual (minoritaria/mayoritaria): {ratio_actual:.4f}")

# ============================================================
# 2. ESTRATEGIA 1: AJUSTE DE THRESHOLD
# ============================================================
print("\n" + "="*80)
print("2. ESTRATEGIA 1: AJUSTE DE THRESHOLD PARA MAXIMIZAR RECALL")
print("="*80)

# Entrenar LightGBM base
lgbm_base = LGBMClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.03,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    class_weight='balanced',
    verbosity=-1
)

print("\n⏳ Entrenando LightGBM base...")
lgbm_base.fit(X_train, y_train, sample_weight=w_train)

# Predecir probabilidades
y_train_proba = lgbm_base.predict_proba(X_train)[:, 1]
y_test_proba = lgbm_base.predict_proba(X_test)[:, 1]
y_dev_proba = lgbm_base.predict_proba(X_dev)[:, 1]

# Buscar mejor threshold
print("\n🔍 Probando diferentes thresholds en TEST:")
print(f"\n{'Threshold':>10} {'Recall':>8} {'Precision':>10} {'F1':>8} {'Accuracy':>10}")
print("-" * 60)

best_threshold = 0.5
best_f1 = 0
best_recall = 0

resultados_threshold = []

for threshold in np.arange(0.3, 0.7, 0.02):
    y_test_pred = (y_test_proba >= threshold).astype(int)
    
    recall = recall_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred, zero_division=0)
    f1 = f1_score(y_test, y_test_pred)
    acc = accuracy_score(y_test, y_test_pred)
    
    resultados_threshold.append({
        'Threshold': threshold,
        'Recall': recall,
        'Precision': precision,
        'F1': f1,
        'Accuracy': acc
    })
    
    print(f"{threshold:>10.2f} {recall:>8.4f} {precision:>10.4f} {f1:>8.4f} {acc:>10.4f}")
    
    # Priorizar recall alto (>0.70) y luego F1
    if recall >= 0.70 and f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_recall = recall

print(f"\n✓ Mejor threshold: {best_threshold:.2f} (F1={best_f1:.4f}, Recall={best_recall:.4f})")

# Evaluar en DEV con mejor threshold
y_dev_pred_optimal = (y_dev_proba >= best_threshold).astype(int)

print(f"\n📊 Resultados en DEV con threshold optimizado ({best_threshold:.2f}):")
print(classification_report(y_dev, y_dev_pred_optimal, 
                          target_names=['Sin anemia', 'Con anemia']))

# ============================================================
# 3. ESTRATEGIA 2: BALANCEO CON SMOTE (CORREGIDO)
# ============================================================
print("\n" + "="*80)
print("3. ESTRATEGIA 2: BALANCEO CON SMOTE + UNDERSAMPLING")
print("="*80)

# Calcular estrategia de balanceo óptima
ratio_objetivo = min(0.9, ratio_actual * 1.5)  # Aumentar hasta 90% o 1.5x el ratio actual

print(f"\n📊 Configuración de balanceo:")
print(f"   Ratio actual: {ratio_actual:.4f}")
print(f"   Ratio objetivo SMOTE: {ratio_objetivo:.4f}")

if ratio_actual >= 0.9:
    print(f"\n⚠️  Las clases ya están bastante balanceadas (ratio={ratio_actual:.4f})")
    print(f"   Se aplicará una estrategia híbrida SMOTE + RandomUnderSampler")
    
    # Estrategia híbrida
    over_sampler = SMOTE(sampling_strategy=0.95, random_state=42)
    under_sampler = RandomUnderSampler(sampling_strategy=0.9, random_state=42)
    
    print(f"\n⏳ Aplicando balanceo híbrido...")
    X_train_balanced, y_train_balanced = over_sampler.fit_resample(X_train, y_train)
    X_train_balanced, y_train_balanced = under_sampler.fit_resample(X_train_balanced, y_train_balanced)
    
else:
    print(f"\n⏳ Aplicando SMOTE...")
    smote = SMOTE(sampling_strategy=ratio_objetivo, random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\n✓ Balanceo aplicado:")
print(f"   Antes: {y_train.value_counts().to_dict()}")
print(f"   Después: {pd.Series(y_train_balanced).value_counts().to_dict()}")

nuevo_ratio = pd.Series(y_train_balanced).value_counts()[1] / pd.Series(y_train_balanced).value_counts()[0]
print(f"   Nuevo ratio: {nuevo_ratio:.4f}")

# Entrenar en datos balanceados
print(f"\n⏳ Entrenando LightGBM con datos balanceados...")
lgbm_balanced = LGBMClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.03,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=-1
)

lgbm_balanced.fit(X_train_balanced, y_train_balanced)

# Evaluar en DEV
y_dev_pred_balanced = lgbm_balanced.predict(X_dev)

print(f"\n📊 Resultados en DEV con datos balanceados:")
print(classification_report(y_dev, y_dev_pred_balanced,
                          target_names=['Sin anemia', 'Con anemia']))

# ============================================================
# 4. ESTRATEGIA 3: CLASS WEIGHT MÁS AGRESIVO
# ============================================================
print("\n" + "="*80)
print("4. ESTRATEGIA 3: CLASS WEIGHT MÁS AGRESIVO")
print("="*80)

# Calcular pesos personalizados
peso_mayoritaria = 1.0
peso_minoritaria = class_counts[0] / class_counts[1] * 1.5  # 50% más peso

class_weights = {0: peso_mayoritaria, 1: peso_minoritaria}

print(f"\n⚙️  Pesos de clase personalizados:")
print(f"   Clase 0 (Sin anemia): {peso_mayoritaria:.2f}")
print(f"   Clase 1 (Con anemia): {peso_minoritaria:.2f}")

lgbm_weighted = LGBMClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.03,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    class_weight=class_weights,
    verbosity=-1
)

print(f"\n⏳ Entrenando modelo con pesos personalizados...")
lgbm_weighted.fit(X_train, y_train, sample_weight=w_train)

y_dev_pred_weighted = lgbm_weighted.predict(X_dev)

print(f"\n📊 Resultados en DEV con class weights personalizados:")
print(classification_report(y_dev, y_dev_pred_weighted,
                          target_names=['Sin anemia', 'Con anemia']))

# ============================================================
# 5. ESTRATEGIA 4: HYPERPARAMETER TUNING
# ============================================================
print("\n" + "="*80)
print("5. ESTRATEGIA 4: HYPERPARAMETER TUNING (GridSearchCV)")
print("="*80)

print("\n⏳ Buscando mejores hiperparámetros...")

# Grid más enfocado en recall
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [6, 8, 10],
    'learning_rate': [0.03, 0.05],
    'num_leaves': [31, 50],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

lgbm_grid = LGBMClassifier(
    random_state=42,
    class_weight='balanced',
    verbosity=-1
)

# Scorer personalizado que prioriza recall
def recall_weighted_scorer(y_true, y_pred):
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return 0.6 * recall + 0.4 * f1

custom_scorer = make_scorer(recall_weighted_scorer)

grid_search = GridSearchCV(
    lgbm_grid,
    param_grid,
    cv=3,
    scoring=custom_scorer,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train, sample_weight=w_train)

print(f"\n✓ Mejores parámetros encontrados:")
for param, value in grid_search.best_params_.items():
    print(f"   {param}: {value}")

# Evaluar mejor modelo
best_model = grid_search.best_estimator_
y_dev_pred_tuned = best_model.predict(X_dev)

print(f"\n📊 Resultados en DEV con hiperparámetros optimizados:")
print(classification_report(y_dev, y_dev_pred_tuned,
                          target_names=['Sin anemia', 'Con anemia']))

# ============================================================
# 6. COMPARACIÓN FINAL
# ============================================================
print("\n" + "="*80)
print("6. COMPARACIÓN DE ESTRATEGIAS")
print("="*80)

estrategias = {
    'Base (threshold=0.5)': lgbm_base.predict(X_dev),
    'Threshold Optimizado': y_dev_pred_optimal,
    'Datos Balanceados': y_dev_pred_balanced,
    'Class Weights Agresivos': y_dev_pred_weighted,
    'Hyperparameter Tuning': y_dev_pred_tuned
}

modelos = {
    'Base (threshold=0.5)': lgbm_base,
    'Threshold Optimizado': lgbm_base,
    'Datos Balanceados': lgbm_balanced,
    'Class Weights Agresivos': lgbm_weighted,
    'Hyperparameter Tuning': best_model
}

resultados_comparacion = []

for nombre, y_pred in estrategias.items():
    modelo = modelos[nombre]
    
    # Obtener probabilidades
    if nombre == 'Threshold Optimizado':
        y_proba = y_dev_proba
    else:
        y_proba = modelo.predict_proba(X_dev)[:, 1]
    
    auc = roc_auc_score(y_dev, y_proba)
    f1 = f1_score(y_dev, y_pred)
    acc = accuracy_score(y_dev, y_pred)
    prec = precision_score(y_dev, y_pred, zero_division=0)
    rec = recall_score(y_dev, y_pred)
    
    # Matriz de confusión
    cm = confusion_matrix(y_dev, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    resultados_comparacion.append({
        'Estrategia': nombre,
        'AUC': auc,
        'F1': f1,
        'Recall': rec,
        'Precision': prec,
        'Accuracy': acc,
        'TP': tp,
        'FN': fn,
        'FP': fp,
        'TN': tn
    })

df_comp = pd.DataFrame(resultados_comparacion).sort_values('Recall', ascending=False)

print(f"\n📊 Tabla comparativa (ordenada por Recall):")
print(df_comp[['Estrategia', 'Recall', 'Precision', 'F1', 'AUC', 'Accuracy']].to_string(index=False))

print(f"\n📊 Detalle de matriz de confusión:")
print(df_comp[['Estrategia', 'TP', 'FN', 'FP', 'TN']].to_string(index=False))

# Identificar mejor estrategia (priorizar recall)
mejor_estrategia_recall = df_comp.iloc[0]['Estrategia']
mejor_estrategia_f1 = df_comp.sort_values('F1', ascending=False).iloc[0]['Estrategia']

print(f"\n🏆 Mejor estrategia por RECALL: {mejor_estrategia_recall}")
print(f"   Recall: {df_comp.iloc[0]['Recall']:.4f}")
print(f"   F1: {df_comp.iloc[0]['F1']:.4f}")

print(f"\n🥈 Mejor estrategia por F1: {mejor_estrategia_f1}")
mejor_f1_row = df_comp[df_comp['Estrategia'] == mejor_estrategia_f1].iloc[0]
print(f"   F1: {mejor_f1_row['F1']:.4f}")
print(f"   Recall: {mejor_f1_row['Recall']:.4f}")

# ============================================================
# 7. ANÁLISIS DE IMPORTANCIA DE FEATURES
# ============================================================
print("\n" + "="*80)
print("7. FEATURE IMPORTANCE DEL MEJOR MODELO")
print("="*80)

# Usar el modelo con mejor F1 para feature importance
modelo_final = modelos[mejor_estrategia_f1]

importances = pd.DataFrame({
    'Feature': features,
    'Importance': modelo_final.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n🔝 Top 15 Features más importantes:")
print(importances.head(15).to_string(index=False))

# ============================================================
# 8. GUARDAR RESULTADOS
# ============================================================
print("\n" + "="*80)
print("8. GUARDANDO RESULTADOS")
print("="*80)

# Guardar Excel
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
archivo_excel = ruta_resultados / f"comparacion_estrategias_{timestamp}.xlsx"

with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
    df_comp.to_excel(writer, sheet_name='Comparacion', index=False)
    importances.to_excel(writer, sheet_name='Feature_Importance', index=False)
    
    # Agregar detalles de thresholds
    df_threshold = pd.DataFrame(resultados_threshold)
    df_threshold.to_excel(writer, sheet_name='Threshold_Analysis', index=False)

print(f"\n✓ Resultados guardados: {archivo_excel}")

# Guardar mejor modelo (según F1)
archivo_modelo = ruta_resultados / f"modelo_optimizado_{timestamp}.pkl"

with open(archivo_modelo, 'wb') as f:
    pickle.dump({
        'modelo': modelo_final,
        'label_encoders': label_encoders,
        'features': features,
        'mejor_estrategia': mejor_estrategia_f1,
        'threshold_optimo': best_threshold,
        'class_weights': class_weights,
        'metricas': mejor_f1_row.to_dict()
    }, f)

print(f"✓ Modelo optimizado guardado: {archivo_modelo}")

print("\n" + "="*80)
print("✅ OPTIMIZACIÓN COMPLETADA")
print("="*80)

print(f"""
📊 RESUMEN EJECUTIVO:

🎯 OBJETIVO: Maximizar RECALL (identificar niños con anemia)

🏆 MEJOR ESTRATEGIA POR RECALL:
   Nombre: {mejor_estrategia_recall}
   Recall: {df_comp.iloc[0]['Recall']:.4f} ({df_comp.iloc[0]['Recall']*100:.2f}%)
   Precision: {df_comp.iloc[0]['Precision']:.4f}
   F1: {df_comp.iloc[0]['F1']:.4f}
   AUC: {df_comp.iloc[0]['AUC']:.4f}

🥈 MEJOR ESTRATEGIA BALANCEADA (F1):
   Nombre: {mejor_estrategia_f1}
   F1: {mejor_f1_row['F1']:.4f}
   Recall: {mejor_f1_row['Recall']:.4f}
   Precision: {mejor_f1_row['Precision']:.4f}

💡 INTERPRETACIÓN:
   - Threshold Optimizado: Ajusta punto de corte para priorizar recall
   - Datos Balanceados: Genera muestras sintéticas para equilibrar clases
   - Class Weights: Penaliza más errores en clase minoritaria
   
📁 ARCHIVOS GENERADOS:
   - {archivo_excel.name}
   - {archivo_modelo.name}

🔬 PRÓXIMOS PASOS:
   1. Revisar matrices de confusión por estrategia
   2. Validar interpretabilidad de features importantes
   3. Evaluar en datos 2023-2024 (validación temporal)
   4. Considerar ensemble de mejores modelos
""")

OPTIMIZACIÓN AVANZADA - PREDICCIÓN DE ANEMIA INFANTIL
Sin HC53 (hemoglobina) para evitar data leakage
Versión Corregida con Ajuste Automático de SMOTE

1. CARGANDO DATOS...

✓ Datos preparados: 17 features
   X_train: (92030, 17)

📊 Distribución de clases en TRAIN:
   Clase 0 (Sin anemia): 49,391 (53.67%)
   Clase 1 (Con anemia): 42,639 (46.33%)
   Ratio actual (minoritaria/mayoritaria): 0.8633

2. ESTRATEGIA 1: AJUSTE DE THRESHOLD PARA MAXIMIZAR RECALL

⏳ Entrenando LightGBM base...

🔍 Probando diferentes thresholds en TEST:

 Threshold   Recall  Precision       F1   Accuracy
------------------------------------------------------------
      0.30   0.9021     0.5388   0.6746     0.5969
      0.32   0.8849     0.5478   0.6767     0.6082
      0.34   0.8678     0.5570   0.6785     0.6190
      0.36   0.8497     0.5653   0.6789     0.6277
      0.38   0.8304     0.5749   0.6794     0.6369
      0.40   0.8113     0.5844   0.6794     0.6452
      0.42   0.7886     0.5927   0.6768     0.651